In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict
import torch

from zipfile import ZipFile
from urllib.request import urlretrieve

from IPython.display import Image

![image.png](attachment:image.png)

In [ ]:
import json

with open('Библиотека/avtostrax/inference/assets/annotations_details_train_panoptic_fake.json', 'r') as f:
    details = json.loads(f.read())
details_codes = [detail['id'] for detail in details['categories']]

In [ ]:
import joblib

# Функция считывания изображений
def read_images(path_to_pkl: str):
    data = joblib.load(path_to_pkl)
    keys = list(data.keys())
    return data, keys

In [ ]:
images_info, paths = read_images("/storage/workflow/Datasets_local/temp_crop_0_1_0/detail_predictions_1639623656.pkl")

In [ ]:
CATEGORIES = [2, 8, 75, 76]
PERCANT_STATE = 15
WINDOW_H = 180
WINDOW_W = int(WINDOW_H * 16 / 9)
STEP = 30

In [ ]:
def cropping_imgs(path: str, mask, key: List[Dict], details_codes: List[int], CATEGORIES: List, CROPP_SIZE: List):
    '''
    Функция для кроппа изображений

    Параметры на вход:
        path: путь до изображения
        mask: маска изображения
        key: кодировка изображений (labeling)
        details_codes: истинная кодировка деталей
        CATEGORIES: Категории (минимум одна), которые должны быть в кроппинге
        CROPP_SIZE: итоговый размер кроппа (после расширения)

    Промежуточные итоги: Нарезка примерных изображений
    '''

    WINDOW_H = int(mask.size()[0] * 0.5) // 9 * 9
    WINDOW_W = int(WINDOW_H * 16 / 9)
    STEP_H = int(WINDOW_H *0.5)
    STEP_W = int(WINDOW_W *0.3)

    new_data = {}

    details_category_indexes = [details_codes.index(category) for category in CATEGORIES]
    needed_key_id = [item['id'] for item in key if item['category_id'] in details_category_indexes]

    PERCANT_STATE = sum([torch.sum(mask == item) for item in needed_key_id])/(mask.shape[0]*mask.shape[1])*125

    # Кодировка раскодировка
    id_to_category_in_key = {item['id']: item['category_id'] for item in key}

    # Увеличение нижней границы (мб поможет с центровкой камеры)
    zeros_tensor = torch.zeros(int(WINDOW_H*0.3), mask.size()[1])
    mask = torch.cat((mask, zeros_tensor), dim=0)

    result_tensors = []
    count_photo = 1

    #подумать над архивом
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = image[:, :, ::-1]

    for i in range(0, mask.shape[0] - WINDOW_H + 1, STEP_H):
        for j in range(0, mask.shape[1] - WINDOW_W + 1, STEP_W):
            # Границы окна
            x_beg, x_end = j, j+WINDOW_W
            y_beg, y_end = i, i+WINDOW_H

            window_mask = mask[y_beg:y_end, x_beg:x_end]
            # Проверка на нахождение нужных категорий на кропе
            check_list = [1 if item in window_mask.unique() else 0 for item in needed_key_id]
            percentages_of_categories_on_cropp = [torch.sum(window_mask == item)/(WINDOW_H*WINDOW_W)*100 for item in needed_key_id]

            percentages_of_background_down = torch.sum(torch.all(window_mask == 0, dim=1))/WINDOW_H*100

            # Условие отбора:
            if window_mask.unique().shape[0] > 3 and sum(check_list) > 1 and 10 < percentages_of_background_down < 50 and sum(percentages_of_categories_on_cropp) > PERCANT_STATE:

                # Создание новой data; пойдет в pkl
                new_key = []
                clone_wm = window_mask.clone()
                for new_id, old_id in enumerate(clone_wm.unique()[1:], start=1):
                    new_key.append({'id': new_id, 'isthing': True, 'category_id': id_to_category_in_key[old_id.item()]})
                    clone_wm[clone_wm == old_id] = new_id


                cropped_img = image[y_beg:y_end, x_beg:x_end]
                resized_cropped_img = cv2.resize(cropped_img, CROPP_SIZE)
                resized_clone_wm = torch.tensor(cv2.resize(clone_wm.numpy(), CROPP_SIZE))

                plt.imshow(resized_cropped_img)
                plt.show()

                plt.imshow(resized_clone_wm)
                plt.show()
                # Изменить size clone_wm
                data_value = (resized_clone_wm, new_key)



In [ ]:
for path in paths:
    mask, key = images_info[path]
    cropping_imgs(path, mask, key, details_codes, WINDOW_H, WINDOW_W, STEP, CATEGORIES, PERCANT_STATE)
    break

In [ ]:
path = "/storage/workflow/Datasets_local/damage_spoiler_0_1_0/cases/"
folders = glob.glob(path + "*/")

# Сортируем папки по дате создания
folders.sort(key=lambda x: os.path.getctime(x))

idx_photo = 0
list_paths = []

for count, folder in enumerate(folders[10:]):
    if count == 20:
        break

    case = folder.split('/')[-2]
    print(f"Case: {case}\n")
    list_paths.extend(glob.glob(f'{folder}*.jpg'))
    for file in glob.glob(f'{folder}*.jpg'):
        print(idx_photo, file)
        image = Image.open(file)
        display(image)
        idx_photo += 1
    print()

In [ ]:
for idx in true_indexes:
    file_name = list_paths[idx]

    folder = file_name.split('/')[-2]

    # Куда копируем
    destination_folder = '/storage/workflow/Datasets_local/damage_spoiler_0_2_0/cases/' + folder + '/'

    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    shutil.copy(file_name, destination_folder)